In [1]:
import data
import models
import cache
import seaborn as sns
import numpy as np
import pandas as pd
import stanity

/home/jacquelineburos/miniconda3/envs/immuneinf/lib/python3.5/site-packages/Cython/Distutils/old_build_ext.py:30: UserWarning: Cython.Distutils.old_build_ext does not properly handle dependencies and is deprecated.
  "Cython.Distutils.old_build_ext does not properly handle dependencies "


In [2]:
sns.set(context='talk')

In [3]:
by = 'cell_type'
sample_n = 500

## get data, as we did in earlier examples

This will help in case we want to compare estimates for particular genes or samples

In [4]:
sample_df = cache.cached(models.prep_sample_df, sample_n=sample_n)

INFO:cache:prep_sample_df: cache_filename set to prep_sample_df.cached.sample_n_500.pkl
INFO:cache:prep_sample_df: Loading result from cache


In [5]:
stan_data = models.prep_stan_data(sample_df, by=by)

## get models from cache

In [7]:
model5 = models.get_model_file(model_name='model5')
fit5 = models.cached_stan_fit(file=model5, data=stan_data, model_name='model5', cache_only=True)

INFO:cache:Step 1: Get compiled model code, possibly from cache
INFO:cache:StanModel: cache_filename set to model5.model_code_5344338527.stanmodel.pkl
INFO:cache:StanModel: Loading result from cache
INFO:cache:Step 2: Get posterior draws from model, possibly from cache
INFO:cache:sampling: cache_filename set to model5.model_code_5344338527.stanfit.chains_81232192355.data_35729880489.iter_31194724242.seed_57902074806.pkl
INFO:cache:sampling: Loading result from cache


In [8]:
model5_1 = models.get_model_file(model_name='model5.1')
fit4 = models.cached_stan_fit(file=model5_1, data=stan_data, model_name='model5.1', cache_only=True)

INFO:cache:Step 1: Get compiled model code, possibly from cache
INFO:cache:StanModel: cache_filename set to model5_1.model_code_65647409730.stanmodel.pkl
INFO:cache:StanModel: Loading result from cache
INFO:cache:Step 2: Get posterior draws from model, possibly from cache
INFO:cache:sampling: cache_filename set to model5_1.model_code_65647409730.stanfit.chains_81232192355.data_35729880489.iter_31194724242.seed_57902074806.pkl
INFO:cache:sampling: Loading result from cache


sampling: Error loading from cache: No module named 'stanfit4model5_1_2933b91e2c7d270e731a7408118d2be1_3429692581183183261'


ValueError: sampling: Cachefile does not exist and cache_only == True. Exiting with failure.

## compute & compare psis-loo for each model

In [13]:
loo5 = cache.cached(stanity.psisloo, log_likelihood=fit5.extract('log_lik')['log_lik'],
                   cache_filename='model5_psisloo.by_{}.sample_n_{}.pkl'.format(by, sample_n),
                   force=False)
loo5.print_summary()

INFO:cache:psisloo: cache_filename set to model5_psisloo.by_cell_type.sample_n_500.pkl
INFO:cache:psisloo: Loading result from cache


greater than 0.5    0.00781
greater than 1      0.00073
dtype: float64

In [ ]:
loo5_1 = cache.cached(stanity.psisloo, log_likelihood=fit5_1.extract('log_lik')['log_lik'],
                   cache_filename='model5_1_psisloo.by_{}.sample_n_{}.pkl'.format(by, sample_n),
                   force=False)
loo5_1.print_summary()

In [ ]:
stanity.loo_compare(loo5, loo5_1)